In [1]:
import numpy as np
import pandas as pd
import catboost as ctb
from sklearn.model_selection import GridSearchCV,train_test_split,RandomizedSearchCV
from sklearn.metrics import f1_score, mean_squared_error, log_loss

In [2]:
data = pd.read_excel("/usr/input/flipr-hackathon-dataset/Train_dataset.xlsx")

In [3]:
data.head()

,people_ID,Region,Gender,Designation,Name,Married,Children,Occupation,Mode_transport,cases/1M,...,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,salary,FT/month,Infect_Prob
0,1,Bhubaneshwar,Female,Mrs,mansi,YES,1.0,Farmer,Public,2,...,93.0,233.0,82.0,58.0,27,7,3600000.0,1300000,2.0,49.135010
1,2,Bhubaneshwar,Female,Mrs,riya masi,YES,2.0,Farmer,Walk,2,...,56.0,328.0,89.0,68.0,5,6,1600000.0,400000,1.0,51.147880
2,3,Bhubaneshwar,Female,Mrs,sunita,NO,1.0,Cleaner,Public,2,...,137.0,213.0,77.0,43.0,40,6,3400000.0,900000,1.0,73.224000
3,4,Bhubaneshwar,Female,Mrs,anjali @ babli,YES,1.0,Driver,Car,2,...,167.0,275.0,64.0,60.0,27,7,700000.0,2300000,1.0,48.779225
4,5,Bhubaneshwar,Female,Mrs,champa karketta,NO,2.0,Manufacturing,Car,2,...,153.0,331.0,71.0,64.0,32,7,3200000.0,1100000,1.0,87.868800


In [4]:
data.isnull().sum()

people_ID                    0
Region                       0
Gender                       0
Designation                  0
Name                        20
Married                      0
Children                   311
Occupation                 747
Mode_transport               3
cases/1M                     0
Deaths/1M                    0
comorbidity                226
Age                          0
Coma score                   0
Pulmonary score              0
cardiological pressure      97
Diuresis                   716
Platelets                  924
HBB                         16
d-dimer                   1114
Heart rate                1114
HDL cholesterol             35
Charlson Index               0
Blood Glucose                0
Insurance                 1090
salary                       0
FT/month                   723
Infect_Prob                  0
dtype: int64

Name, Designation are not important

In [5]:
data = data.drop(["Name", "Designation",],axis = 1)
data = data.loc[~data.iloc[:,[1,2,3,5,6,9,12,13,]].isnull().any(axis=1)]
y = data["Infect_Prob"]
data = data.drop(["Infect_Prob"], axis = 1)
#y = (y >= 50).astype("float64")
y /= 100

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data.values, y, random_state = 42)

In [7]:
X_train[0]

array([14832, 'Chandigarh', 'Male', 'YES', 1.0, 'Manufacturing', 'Public',
       2, 0, 'None', 41, 10, '<100', 'Stage-02', 324.0, 140.0, 80.0,
       323.0, 84.0, 45.0, 17, 4, 600000.0, 400000, 1.0], dtype=object)

In [8]:
cat_features = [1,2,3,5,6,9,12,13,]

In [9]:
clf = ctb.CatBoostRegressor(boosting_type="Plain",loss_function="RMSE",learning_rate=0.01,border_count=254,verbose=0,n_estimators=10000,rsm=0.4,cat_features = cat_features, subsample = 0.6)

In [10]:
params = {"depth": [6, 8, 10, 12,],}
grid = GridSearchCV(estimator = clf, param_grid = params, cv = 5, verbose = 2,scoring="neg_mean_squared_error")
grid.fit(X_train, y_train, eval_set = [(X_test, y_test)], early_stopping_rounds = 10,)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] depth=6 .........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......................................... depth=6, total=   1.9s
[CV] depth=6 .........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s


[CV] .......................................... depth=6, total=   1.7s
[CV] depth=6 .........................................................
[CV] .......................................... depth=6, total=   2.2s
[CV] depth=6 .........................................................
[CV] .......................................... depth=6, total=   1.9s
[CV] depth=6 .........................................................
[CV] .......................................... depth=6, total=   1.5s
[CV] depth=8 .........................................................
[CV] .......................................... depth=8, total=   3.2s
[CV] depth=8 .........................................................
[CV] .......................................... depth=8, total=   2.7s
[CV] depth=8 .........................................................
[CV] .......................................... depth=8, total=   3.0s
[CV] depth=8 .........................................................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f3f04ac83c8>,
             iid='deprecated', n_jobs=None,
             param_grid={'depth': [6, 8, 10, 12]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=2)

In [11]:
grid.best_params_

{'depth': 10}

In [12]:
grid.best_score_

-0.006318172028984102

In [13]:
mean_squared_error(y_test, grid.best_estimator_.predict(X_test))

0.00644555916845476

Now, tuning the regularisation parameter:

In [14]:
clf = ctb.CatBoostRegressor(boosting_type="Plain", loss_function="RMSE", learning_rate=0.01, border_count=254,\
                            verbose=0, n_estimators=10000, rsm=0.4,\
                            cat_features = cat_features, \
                            subsample = 0.6, depth = 10)

params = {"l2_leaf_reg": [0.01, 0.1, 1, 3, 10, 15,],}
grid = GridSearchCV(estimator = clf, param_grid = params, cv = 5, verbose = 2,scoring="neg_mean_squared_error")
grid.fit(X_train, y_train, eval_set = [(X_test, y_test)], early_stopping_rounds = 10,)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] l2_leaf_reg=0.01 ................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................................. l2_leaf_reg=0.01, total=   4.1s
[CV] l2_leaf_reg=0.01 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.1s remaining:    0.0s


[CV] ................................. l2_leaf_reg=0.01, total=   3.0s
[CV] l2_leaf_reg=0.01 ................................................
[CV] ................................. l2_leaf_reg=0.01, total=   4.1s
[CV] l2_leaf_reg=0.01 ................................................
[CV] ................................. l2_leaf_reg=0.01, total=   2.6s
[CV] l2_leaf_reg=0.01 ................................................
[CV] ................................. l2_leaf_reg=0.01, total=   3.6s
[CV] l2_leaf_reg=0.1 .................................................
[CV] .................................. l2_leaf_reg=0.1, total=   3.0s
[CV] l2_leaf_reg=0.1 .................................................
[CV] .................................. l2_leaf_reg=0.1, total=   4.4s
[CV] l2_leaf_reg=0.1 .................................................
[CV] .................................. l2_leaf_reg=0.1, total=   3.8s
[CV] l2_leaf_reg=0.1 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  2.2min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f3f0408d940>,
             iid='deprecated', n_jobs=None,
             param_grid={'l2_leaf_reg': [0.01, 0.1, 1, 3, 10, 15]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=2)

In [15]:
grid.best_params_

{'l2_leaf_reg': 1}

In [16]:
grid.best_score_

-0.006299534706695709

In [17]:
mean_squared_error(y_test, grid.best_estimator_.predict(X_test))

0.006408008840482957

In [18]:
clf = ctb.CatBoostRegressor(boosting_type="Plain", loss_function="RMSE", learning_rate=0.01, border_count=254,\
                            verbose=0, n_estimators=10000, rsm=0.4,\
                            cat_features = cat_features, \
                            subsample = 0.6, depth = 10, l2_leaf_reg = 1)

params = {"random_strength": [0.1,0.5,1,2]}
grid = GridSearchCV(estimator = clf, param_grid = params, cv = 5, verbose = 2,scoring="neg_mean_squared_error")
grid.fit(X_train, y_train, eval_set = [(X_test, y_test)], early_stopping_rounds = 10,)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] random_strength=0.1 .............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .............................. random_strength=0.1, total=   3.9s
[CV] random_strength=0.1 .............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.9s remaining:    0.0s


[CV] .............................. random_strength=0.1, total=   4.2s
[CV] random_strength=0.1 .............................................
[CV] .............................. random_strength=0.1, total=   3.4s
[CV] random_strength=0.1 .............................................
[CV] .............................. random_strength=0.1, total=   4.3s
[CV] random_strength=0.1 .............................................
[CV] .............................. random_strength=0.1, total=   3.6s
[CV] random_strength=0.5 .............................................
[CV] .............................. random_strength=0.5, total=   3.8s
[CV] random_strength=0.5 .............................................
[CV] .............................. random_strength=0.5, total=   4.1s
[CV] random_strength=0.5 .............................................
[CV] .............................. random_strength=0.5, total=   4.7s
[CV] random_strength=0.5 .............................................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f3f0408def0>,
             iid='deprecated', n_jobs=None,
             param_grid={'random_strength': [0.1, 0.5, 1, 2]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=2)

In [19]:
grid.best_params_

{'random_strength': 1}

In [20]:
grid.best_score_

-0.006299534706695709

In [21]:
mean_squared_error(y_test, grid.best_estimator_.predict(X_test))

0.006408008840482957

In [22]:
clf = ctb.CatBoostRegressor(boosting_type="Plain", loss_function="RMSE", learning_rate=0.01, border_count=254,\
                            verbose=0, n_estimators=10000,\
                            cat_features = cat_features, \
                            subsample = 0.6, depth = 10, l2_leaf_reg = 1)

params = {"rsm": [0.1,0.2,0.4,0.6,0.8,1]}
grid = GridSearchCV(estimator = clf, param_grid = params, cv = 5, verbose = 2,scoring="neg_mean_squared_error")
grid.fit(X_train, y_train, eval_set = [(X_test, y_test)], early_stopping_rounds = 10,)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] rsm=0.1 .........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......................................... rsm=0.1, total=   1.8s
[CV] rsm=0.1 .........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV] .......................................... rsm=0.1, total=   2.0s
[CV] rsm=0.1 .........................................................
[CV] .......................................... rsm=0.1, total=   2.0s
[CV] rsm=0.1 .........................................................
[CV] .......................................... rsm=0.1, total=   1.9s
[CV] rsm=0.1 .........................................................
[CV] .......................................... rsm=0.1, total=   2.1s
[CV] rsm=0.2 .........................................................
[CV] .......................................... rsm=0.2, total=   2.9s
[CV] rsm=0.2 .........................................................
[CV] .......................................... rsm=0.2, total=   2.7s
[CV] rsm=0.2 .........................................................
[CV] .......................................... rsm=0.2, total=   2.9s
[CV] rsm=0.2 .........................................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  2.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f3f04093080>,
             iid='deprecated', n_jobs=None,
             param_grid={'rsm': [0.1, 0.2, 0.4, 0.6, 0.8, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=2)

In [23]:
grid.best_params_

{'rsm': 1}

In [24]:
grid.best_score_

-0.006295227289295946

In [25]:
mean_squared_error(y_test, grid.best_estimator_.predict(X_test))

0.006424068326381498

In [26]:
clf = ctb.CatBoostRegressor(boosting_type="Plain", loss_function="RMSE", learning_rate=0.01, border_count=254,\
                            verbose=0, n_estimators=10000,\
                            cat_features = cat_features, \
                            depth = 10, l2_leaf_reg = 1)

params = {"subsample": [0.1,0.2,0.4,0.6,0.8,1]}
grid = GridSearchCV(estimator = clf, param_grid = params, cv = 5, verbose = 2,scoring="neg_mean_squared_error")
grid.fit(X_train, y_train, eval_set = [(X_test, y_test)], early_stopping_rounds = 10,)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] subsample=0.1 ...................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................................... subsample=0.1, total=   9.4s
[CV] subsample=0.1 ...................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.5s remaining:    0.0s


[CV] .................................... subsample=0.1, total=   7.5s
[CV] subsample=0.1 ...................................................
[CV] .................................... subsample=0.1, total=  10.2s
[CV] subsample=0.1 ...................................................
[CV] .................................... subsample=0.1, total=   8.4s
[CV] subsample=0.1 ...................................................
[CV] .................................... subsample=0.1, total=   7.3s
[CV] subsample=0.2 ...................................................
[CV] .................................... subsample=0.2, total=   7.2s
[CV] subsample=0.2 ...................................................
[CV] .................................... subsample=0.2, total=   6.4s
[CV] subsample=0.2 ...................................................
[CV] .................................... subsample=0.2, total=   6.7s
[CV] subsample=0.2 ...................................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  4.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f3f04093a58>,
             iid='deprecated', n_jobs=None,
             param_grid={'subsample': [0.1, 0.2, 0.4, 0.6, 0.8, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=2)

In [27]:
grid.best_params_

{'subsample': 0.6}

In [28]:
grid.best_score_

-0.006295227289295946

In [29]:
mean_squared_error(y_test, grid.best_estimator_.predict(X_test))

0.006424068326381498